In [ ]:
## Part I: Preprocessing and EDA
- The data comes from a global e-retailer company, including orders from 2012 to 2015. 
Import the **Orders** dataset and do some basic EDA. 
- For problem 1 to 3, we mainly focus on data cleaning and data visualizations. 
You can use all the packages that you are familiar with to conduct some plots and 
also provide **brief interpretations** about your findings.

### Problem 1: Dataset Import & Cleaning
Check **"Profit"** and **"Sales"** in the dataset, convert these two columns to numeric type. 
### Problem 2: Inventory Management
- Retailers that depend on seasonal shoppers have a particularly challenging job when it comes 
to inventory management. Your manager is making plans for next year's inventory.
- He wants you to answer the following questions:
    1. Is there any seasonal trend of inventory in the company?
    2. Is the seasonal trend the same for different categories?
- ***Hint:*** For each order, it has an attribute called `Quantity` that indicates the number 
    of product in the order. If an order contains more than one product, there will be multiple 
    observations of the same order.
### Problem 3: Why did customers make returns?
- Your manager required you to give a brief report (**Plots + Interpretations**) on returned orders.

	1. How much profit did we lose due to returns each year?
	2. How many customer returned more than once? more than 5 times?
	3. Which regions are more likely to return orders?
	4. Which categories (sub-categories) of products are more likely to be returned?
- ***Hint:*** Merge the **Returns** dataframe with the **Orders** dataframe using `Order.ID`.

In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats

orders = pd.read_csv('data/Orders.csv')
returns = pd.read_csv('data/Returns.csv')
pd.options.display.max_columns = None

In [3]:
orders.columns

Index(['Row.ID', 'Order.ID', 'Order.Date', 'Ship.Date', 'Ship.Mode',
       'Customer.ID', 'Customer.Name', 'Segment', 'Postal.Code', 'City',
       'State', 'Country', 'Region', 'Market', 'Product.ID', 'Category',
       'Sub.Category', 'Product.Name', 'Sales', 'Quantity', 'Discount',
       'Profit', 'Shipping.Cost', 'Order.Priority'],
      dtype='object')

In [4]:
returns.columns

Index(['Returned', 'Order ID', 'Region'], dtype='object')

In [9]:
orders.sample(10)


,Row.ID,Order.ID,Order.Date,Ship.Date,Ship.Mode,Customer.ID,Customer.Name,Segment,Postal.Code,City,State,Country,Region,Market,Product.ID,Category,Sub.Category,Product.Name,Sales,Quantity,Discount,Profit,Shipping.Cost,Order.Priority
26639,30847,ID-2013-KH1669092-41601,11/23/13,11/28/13,Standard Class,KH-1669092,Kristen Hastings,Corporate,NaN,Manukau City,Auckland,New Zealand,Oceania,Asia Pacific,OFF-SU-2995,Office Supplies,Supplies,"Acme Shears, Serrated",$111.89,4,0.400,-$54.19,7.210,Medium
50102,50375,NI-2012-SP1086095-41090,6/30/12,7/3/12,First Class,SP-1086095,Sung Pak,Corporate,NaN,Ibadan,Oyo,Nigeria,Western Africa,Africa,OFF-BI-2917,Office Supplies,Binders,"Acco Index Tab, Clear",$5.15,2,0.700,-$9.31,1.190,Critical
37054,21973,IN-2015-ES1402027-42064,3/1/15,3/5/15,Standard Class,ES-1402027,Erica Smith,Consumer,NaN,Nantong,Jiangsu,China,Eastern Asia,Asia Pacific,OFF-EN-4432,Office Supplies,Envelopes,"GlobeWeis Business Envelopes, Security-Tint",$39.48,2,0.000,$4.32,2.990,High
4980,9024,MX-2013-BN1151531-41552,10/5/13,10/9/13,Standard Class,BN-1151531,Bradley Nguyen,Consumer,NaN,Santiago de Cuba,Santiago de Cuba,Cuba,Caribbean,LATAM,TEC-CO-3709,Technology,Copiers,"Canon Wireless Fax, High-Speed","$1,002.95",4,0.002,$168.79,66.937,Medium
38311,39578,CA-2015-MC17605140-42134,5/10/15,5/11/15,First Class,MC-176051408,Matt Connell,Corporate,28540.0,Jacksonville,North Carolina,United States,Southern US,USCA,FUR-CH-4556,Furniture,Chairs,Harbour Creations Steel Folding Chair,$207.00,3,0.200,$25.88,63.050,Critical
24275,30535,IN-2013-AP1072092-41409,5/15/13,5/17/13,Second Class,AP-1072092,Anne Pryor,Home Office,NaN,New Plymouth,Taranaki,New Zealand,Oceania,Asia Pacific,OFF-EN-4904,Office Supplies,Envelopes,"Jiffy Business Envelopes, Security-Tint",$31.92,2,0.000,$6.06,8.700,Critical
24746,4964,US-2015-JC1538598-42102,4/8/15,4/12/15,Standard Class,JC-1538598,Jenna Caffey,Consumer,NaN,David,Chiriquí,Panama,Central America,LATAM,TEC-PH-3150,Technology,Phones,"Apple Speaker Phone, Cordless",$151.38,3,0.400,-$90.84,8.376,Medium
30958,24997,IN-2014-DP1339058-41880,8/29/14,9/2/14,Standard Class,DP-1339058,Dennis Pardue,Home Office,NaN,Kanpur,Uttar Pradesh,India,Southern Asia,Asia Pacific,OFF-SU-4995,Office Supplies,Supplies,"Kleencut Trimmer, Serrated",$114.39,3,0.000,$14.85,5.080,Medium
14811,916,MX-2012-DC1328582-41026,4/27/12,5/1/12,Standard Class,DC-1328582,Debra Catini,Consumer,NaN,Soledad Díez Gutiérrez,San Luis Potosí,Mexico,Central America,LATAM,TEC-PH-3140,Technology,Phones,"Apple Office Telephone, Full Size",$177.52,4,0.000,$7.04,20.052,High
1239,28645,IN-2012-BM1178558-41124,8/3/12,8/7/12,Standard Class,BM-1178558,Bryan Mills,Consumer,NaN,Tiruchchirappalli,Tamil Nadu,India,Southern Asia,Asia Pacific,FUR-BO-3639,Furniture,Bookcases,"Bush Library with Doors, Metal","$1,455.12",4,0.000,$116.40,179.620,Medium


In [10]:
returns.sample(10)

,Returned,Order ID,Region
1022,Yes,EG-2014-CS186038-41690,North Africa
444,Yes,IT-2014-TP2141545-41858,Western Europe
875,Yes,CA-2014-TB21280140-41786,Eastern US
532,Yes,SF-2015-RS9765117-42224,Southern Africa
232,Yes,CA-2012-BS11755140-41046,Eastern US
424,Yes,CA-2015-KH16330140-42329,Eastern US
351,Yes,BU-2012-EH394519-41265,Eastern Europe
956,Yes,CA-2015-HM14860140-42221,Western US
962,Yes,US-2014-LH1675055-41840,Central America
179,Yes,ES-2014-LC1705048-41718,Western Europe
